In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torch.optim import Adam

import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from models import SimpleNet

In [2]:
%config Completer.use_jedi = False

In [3]:
TRAIN_DATA_PATH = "train.csv"
TEST_DATA_PATH = "test.csv"

In [4]:
class KannadaDataset(Dataset):
    def __init__(self, mode="train", reshape=True):
        if mode=="train":
            data_raw = pd.read_csv(TRAIN_DATA_PATH)
            first_column = "label"
        elif mode=="test":
            data_raw = pd.read_csv(TEST_DATA_PATH)
            first_column = "id"
        labels = torch.tensor(data_raw[first_column].to_numpy())
        pixels = torch.tensor(data_raw.iloc[:, 1:].to_numpy()/255)
        if reshape:
            pixels = pixels.reshape(-1, 28, 28)
        self.pixels = pixels
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.pixels[idx]
        label = self.labels[idx]
        sample = {"image": image, "label": label}

        return sample

In [5]:
train_data, valid_data = random_split(KannadaDataset(mode="train"), [55000, 5000])

test_data = KannadaDataset(mode="test")

In [6]:
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=256)

In [7]:
model = SimpleNet(10).to("cuda")

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [9]:
for epoch in range(8):
    running_loss = 0.0
    for num_batch, batch in enumerate(train_loader):
        images, labels = batch['image'], batch['label']
        images = images.unsqueeze(1).to(torch.float32).to("cuda")
        labels = labels.to("cuda")
        out = model(images)
        loss = criterion(out, labels)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item()
        if num_batch%100==99:
            correct = 0
            total = 0 
            with torch.no_grad():
                for i, batch in enumerate(valid_loader):
                    images, labels = batch['image'], batch['label']
                    images = images.unsqueeze(1).to(torch.float32).to("cuda")
                    labels = labels.to("cuda")
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            print(f"Epoch: {epoch}, Batch: {num_batch}, Loss {running_loss/100}, Val acc {correct/total}")
            running_loss = 0.0
print("Training done!")

Epoch: 0, Batch: 99, Loss 0.1652186479140073, Val acc 0.993
Epoch: 0, Batch: 199, Loss 0.026915152128785848, Val acc 0.994
Epoch: 1, Batch: 99, Loss 0.0161675859359093, Val acc 0.9962
Epoch: 1, Batch: 199, Loss 0.021237963701132685, Val acc 0.997
Epoch: 2, Batch: 99, Loss 0.015101826512254774, Val acc 0.995
Epoch: 2, Batch: 199, Loss 0.016148616882856005, Val acc 0.9962
Epoch: 3, Batch: 99, Loss 0.012434139280812814, Val acc 0.9974
Epoch: 3, Batch: 199, Loss 0.010660937725915573, Val acc 0.9964
Epoch: 4, Batch: 99, Loss 0.010894121187739074, Val acc 0.993
Epoch: 4, Batch: 199, Loss 0.011358702271827496, Val acc 0.996
Epoch: 5, Batch: 99, Loss 0.006949024936475325, Val acc 0.9966
Epoch: 5, Batch: 199, Loss 0.010386780123808422, Val acc 0.9972
Epoch: 6, Batch: 99, Loss 0.008054018333205022, Val acc 0.9978
Epoch: 6, Batch: 199, Loss 0.004852478063985473, Val acc 0.9972
Epoch: 7, Batch: 99, Loss 0.0069126790435984735, Val acc 0.997
Epoch: 7, Batch: 199, Loss 0.006517529205157188, Val acc 0

In [10]:
torch.save(model.state_dict(), "kannada_mnist_v7.pth")

In [11]:
correct = 0
total = 0 
with torch.no_grad():
    for i, batch in enumerate(valid_loader):
        images, labels = batch['image'], batch['label']
        images = images.unsqueeze(1).to(torch.float32).to("cuda")
        labels = labels.to("cuda")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print('Accuracy of the network on the 10000 test images: %s' % (
    correct / total))

Accuracy of the network on the 10000 test images: 0.9966


In [12]:
# model.load_state_dict(torch.load("kannada_mnist_v5.pth"))

In [13]:
res = []
with torch.no_grad():
    for batch in test_loader:
        images, ids = batch['image'], batch['label']
        images = images.unsqueeze(1).to(torch.float32).to("cuda")
        ids = ids.to("cuda")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        res += zip(ids.tolist(), predicted.tolist())
        
len(res)

5000

In [14]:
with open("submission5.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)
    # header
    writer.writerow(["id", "label"])
    writer.writerows(res)